In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import ast
import joblib
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
d_1= pd.read_csv('movie/movies_metadata.csv')
d_2 = pd.read_csv('movie/keywords.csv')
d_3= pd.read_csv('movie/credits.csv')

/tmp/ipykernel_5797/2316738319.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  d_1= pd.read_csv('movie/movies_metadata.csv')


In [3]:
d_1.head(3)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [4]:
df = d_1[['id', 'title', 'genres', 'overview', 'release_date']]
df.head(1)

,id,title,genres,overview,release_date
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",1995-10-30


In [5]:
df['keywords'] = d_2['keywords']
df.head(1)

/tmp/ipykernel_5797/1970470473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = d_2['keywords']


,id,title,genres,overview,release_date,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",1995-10-30,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [6]:
df['crew'] = d_3['crew']
df.head(10)


/tmp/ipykernel_5797/3928100396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crew'] = d_3['crew']


,id,title,genres,overview,release_date,keywords,crew
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",1995-10-30,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,1995-12-15,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,1995-12-22,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",1995-12-22,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,1995-02-10,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
5,949,Heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","Obsessive master thief, Neil McCauley leads a ...",1995-12-15,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
6,11860,Sabrina,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",An ugly duckling having undergone a remarkable...,1995-12-15,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...","[{'credit_id': '52fe44959251416c75039da9', 'de..."
7,45325,Tom and Huck,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","A mischievous young boy, Tom Sawyer, witnesses...",1995-12-22,[],"[{'credit_id': '52fe46bdc3a36847f810f797', 'de..."
8,9091,Sudden Death,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",International action superstar Jean Claude Van...,1995-12-22,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de..."
9,710,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",James Bond must unmask the mysterious head of ...,1995-11-16,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            45466 non-null  object
 1   title         45460 non-null  object
 2   genres        45466 non-null  object
 3   overview      44512 non-null  object
 4   release_date  45379 non-null  object
 5   keywords      45466 non-null  object
 6   crew          45466 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [8]:
df['genres'] = df['genres'].replace('[]', pd.NA)
df['keywords'] = df['keywords'].replace('[]', pd.NA)
df['crew'] = df['crew'].replace('[]', pd.NA)

/tmp/ipykernel_5797/1534052240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].replace('[]', pd.NA)
/tmp/ipykernel_5797/1534052240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].replace('[]', pd.NA)
/tmp/ipykernel_5797/1534052240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [9]:
df.isna().sum()

id                  0
title               6
genres           2442
overview          954
release_date       87
keywords        14325
crew              771
dtype: int64

In [10]:
df_1 = df.dropna()

In [11]:
df_2 = df_1.drop_duplicates()

In [12]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29327 entries, 0 to 45463
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            29327 non-null  object
 1   title         29327 non-null  object
 2   genres        29327 non-null  object
 3   overview      29327 non-null  object
 4   release_date  29327 non-null  object
 5   keywords      29327 non-null  object
 6   crew          29327 non-null  object
dtypes: object(7)
memory usage: 1.8+ MB


In [13]:
df_2

,id,title,genres,overview,release_date,keywords,crew
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",1995-10-30,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,1995-12-15,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,1995-12-22,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",1995-12-22,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,1995-02-10,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...
45439,84710,Jungle Woman,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",Paula the ape woman (Acquanetta) is alive and ...,1944-06-01,"[{'id': 1491, 'name': 'entertainer'}, {'id': 2...","[{'credit_id': '52fe46019251416c91045d65', 'de..."
45441,14008,Cadet Kelly,"[{'id': 35, 'name': 'Comedy'}]",Hyperactive teenager Kelly is enrolled into a ...,2002-03-07,"[{'id': 6038, 'name': 'marriage'}, {'id': 1018...","[{'credit_id': '5704afecc3a3681f0c000315', 'de..."
45445,49277,The Hilarious Posters,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",A wall full of advertising posters comes to life.,1906-01-01,"[{'id': 188, 'name': 'buddhism'}, {'id': 1536,...","[{'credit_id': '52fe46a2c3a368484e09b639', 'de..."
45449,14885,Pooh's Heffalump Halloween Movie,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","It's Halloween in the 100 Acre Wood, and Roo's...",2005-09-13,"[{'id': 6133, 'name': 'industry'}, {'id': 7337...","[{'credit_id': '52fe49099251416c910a0a89', 'de..."


In [14]:
def converter(obj):
    list = []
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list

In [15]:
df_2['genres']= df_2['genres'].apply(converter)
df_2['keywords']= df_2['keywords'].apply(converter)


/tmp/ipykernel_5797/1318875646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['genres']= df_2['genres'].apply(converter)
/tmp/ipykernel_5797/1318875646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['keywords']= df_2['keywords'].apply(converter)


In [16]:
def converter1(obj):
    list = []
    for i in ast.literal_eval(obj):
        if i['department'] == 'Directing':
            list.append(i['name'])
    return list

In [17]:
df_2['crew']= df_2['crew'].apply(converter1)

/tmp/ipykernel_5797/550323721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['crew']= df_2['crew'].apply(converter1)


In [18]:

def date_convert(fulldate):

    return [int(fulldate.split('-')[0])]

df_2['release_date'] = df_2['release_date'].apply(date_convert)
# df_2['release_date'] = df_2['release_date'].str[:4]

/tmp/ipykernel_5797/953536363.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['release_date'] = df_2['release_date'].apply(date_convert)


In [19]:
df_2

,id,title,genres,overview,release_date,keywords,crew
0,862,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",[1995],"[jealousy, toy, boy, friendship, friends, riva...","[John Lasseter, Roman Figun, Desirée Mourad]"
1,8844,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,[1995],"[board game, disappearance, based on children'...",[Joe Johnston]
2,15602,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,[1995],"[fishing, best friend, duringcreditsstinger, o...",[Howard Deutch]
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",[1995],"[based on novel, interracial relationship, sin...",[Forest Whitaker]
4,11862,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,[1995],"[baby, midlife crisis, confidence, aging, daug...",[Charles Shyer]
...,...,...,...,...,...,...,...
45439,84710,Jungle Woman,"[Drama, Fantasy, Horror]",Paula the ape woman (Acquanetta) is alive and ...,[1944],"[entertainer, russia, musical, talent show, as...",[Frank Theys]
45441,14008,Cadet Kelly,[Comedy],Hyperactive teenager Kelly is enrolled into a ...,[2002],"[marriage, based on play or musical]",[Robert Gardner]
45445,49277,The Hilarious Posters,"[Fantasy, Comedy]",A wall full of advertising posters comes to life.,[1906],"[buddhism, religious education, buddha, religion]",[Oleg Assadulin]
45449,14885,Pooh's Heffalump Halloween Movie,"[Animation, Family]","It's Halloween in the 100 Acre Wood, and Roo's...",[2005],"[industry, business ethics, crew, blue collar,...",[Reginald Le Borg]


In [20]:
df_2['tags'] = df_2['genres'] + df_2['keywords'] + df_2['crew'] + df_2['release_date']

/tmp/ipykernel_5797/1995438862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['tags'] = df_2['genres'] + df_2['keywords'] + df_2['crew'] + df_2['release_date']


In [21]:
# remove space fro names
df_2['tags']= df_2['tags'].apply(lambda x: [str(i).replace(' ', '') for i in x])

# convert to paragraphe
df_2['tags']= df_2['tags'].apply(lambda x:' '.join(x))

/tmp/ipykernel_5797/3716274457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['tags']= df_2['tags'].apply(lambda x: [str(i).replace(' ', '') for i in x])
/tmp/ipykernel_5797/3716274457.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['tags']= df_2['tags'].apply(lambda x:' '.join(x))


In [22]:
df_2['description'] = df_2['overview'] + ' ' + df_2['tags']

#remove -- from text
df_2['description'] = df_2['description'].replace('-', '', regex=False)

# convert to lower case
df_2['description'] = df_2['description'].apply(lambda x:x.lower())



/tmp/ipykernel_5797/1811348960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['description'] = df_2['overview'] + ' ' + df_2['tags']
/tmp/ipykernel_5797/1811348960.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['description'] = df_2['description'].replace('-', '', regex=False)
/tmp/ipykernel_5797/1811348960.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [23]:
ps = PorterStemmer()

In [24]:
def d_con(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return ' '.join(l)

In [25]:
df_2['description'] = df_2['description'].apply(d_con)

/tmp/ipykernel_5797/2476239127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['description'] = df_2['description'].apply(d_con)


In [26]:
movie = df_2[['id', 'title', 'description']]

In [27]:
movie

,id,title,description
0,862,Toy Story,"led by woody, andy' toy live happili in hi roo..."
1,8844,Jumanji,when sibl judi and peter discov an enchant boa...
2,15602,Grumpier Old Men,a famili wed reignit the ancient feud between ...
3,31357,Waiting to Exhale,"cheat on, mistreat and step on, the women are ..."
4,11862,Father of the Bride Part II,just when georg bank ha recov from hi daughter...
...,...,...,...
45439,84710,Jungle Woman,paula the ape woman (acquanetta) is aliv and w...
45441,14008,Cadet Kelly,hyperact teenag kelli is enrol into a militari...
45445,49277,The Hilarious Posters,a wall full of advertis poster come to life. f...
45449,14885,Pooh's Heffalump Halloween Movie,"it' halloween in the 100 acr wood, and roo' be..."


In [36]:
cv = CountVectorizer(max_features=35000, stop_words='english')
vector_data = cv.fit_transform(movie['description']).toarray()

In [37]:
sim = cosine_similarity(vector_data)

In [38]:
# get title
# movie[movie['title']== 'Avatar'].index[0]


#short sim
sorted(list(enumerate(sim[0])), reverse=True, key=lambda x:x[1])[0:11]

# 30000 => 0.9999999999999998
# 35000 => 1.0

# movie.iloc[0].title

[(0, 1.0),
 (2610, 0.4364357804719848),
 (12774, 0.347689743729574),
 (18709, 0.30000000000000004),
 (28190, 0.25869823880548926),
 (26939, 0.2483498169496957),
 (544, 0.24157059753032908),
 (20664, 0.23959317875170782),
 (26091, 0.23094010767585027),
 (177, 0.22978625750451448),
 (28283, 0.22754551365455156)]

In [31]:
def recommended(title):
    m_index = movie[movie['title']== title].index[0]
    movie_list = sorted(list(enumerate(sim[m_index])), reverse=True, key=lambda x:x[1])[1:11]
    

    # for i in movie_list:
    #     print(movie.iloc[i[0]].title)
    return movie_list

 


In [32]:
recommended('Toy Story')

[(2610, 0.41344911529736156),
 (6187, 0.3662948371965041),
 (28190, 0.3528251645047713),
 (26939, 0.33407655239053047),
 (20664, 0.3304135690539768),
 (7886, 0.3292763870490873),
 (544, 0.32850915221752475),
 (18503, 0.3207501495497921),
 (177, 0.3205079825469411),
 (4121, 0.3164751088516942)]

In [ ]:
recommendations = {}
top_k = 20
for idx, row in enumerate(sim):
    top_indices = row.argsort()[::-1][1:top_k+1] 
    recommendations[movie['title'].iloc[idx]] = [movie['title'].iloc[i] for i in top_indices]


joblib.dump(recommendations, 'model20.pkl')




['top_recommendations.pkl']

In [34]:
joblib.dump(movie, 'movie_data.pkl')
joblib.dump(sim, 'model.pkl')

OSError: [Errno 75] Value too large for defined data type: 'model.pkl'